In [45]:
import networkx as nx
import numpy as np
import scipy as sp
import scipy.linalg as lin

import common.generators as gen

# Get graph and adj matrix
# so far, snowflakecycle(5,5,3) is the only setting that produces a deceptive graph.
G = gen.snowflakecycle(5,5,3)
AG = nx.to_numpy_matrix(G)

# Build Walk-submatrix
inds = [0,1,2]
Ut = np.zeros((len(inds),3))

A_temp = AG**2
diag = np.diag(A_temp)
Ut[:,0] = np.squeeze(diag[inds])

A_temp = AG**3
diag = np.diag(A_temp)
Ut[:,1] = np.squeeze(diag[inds])

A_temp = AG**5
diag = np.diag(A_temp)
Ut[:,2] = np.squeeze(diag[inds])

Ut = np.matrix(Ut)


# Construct Linear Program
num_rows, num_cols = Ut.shape

A_eq = np.zeros( (num_rows, num_cols+2) )
A_eq[:,0:num_cols] = Ut
A_eq[:,num_cols] = -np.squeeze(np.ones(shape=(num_rows,1)))
A_eq[:,-1] = -np.squeeze(np.zeros(shape=(num_rows,1)))
A_eq = np.matrix(A_eq)

num_rows,num_cols = np.shape(A_eq)
c = np.ones(num_cols)

A_ub = np.zeros( (num_rows+1, num_cols) )
A_ub[:,0:num_rows+1] = -np.identity(num_rows+1)
A_ub[0:3,-1] = np.squeeze(-np.ones(shape=(num_rows,1)))
A_ub = np.matrix(A_ub)

# Get diagonal of expm(A)
evals, evec = lin.eigh(AG)
exp_vals = [np.exp(val) for val in evals]
EXPM_AG = evec*(np.asmatrix(np.diag(exp_vals)))*evec.T
d = np.diag(EXPM_AG)
g = [d[idx] for idx in inds]
b_eq = -np.asarray(g)

b_ub = np.zeros(num_rows+1)
b_ub[-1] = -max(abs(b_eq))

# Return result
opt_obj = sp.optimize.linprog(
    c=c,
    A_ub=A_ub,
    b_ub=b_ub,
    A_eq=A_eq,
    b_eq=b_eq
)
opt_obj
# Successful termination means we have constructed a deceptive function

# Construct deceptive function using coefficients from the optimization problem
x = opt_obj.x
coeffs = np.asmatrix(x[0:3])
coeffs = coeffs.T

final_diag = Ut*coeffs + np.asmatrix(g).T
print(final_diag)

[[ 11.63965846]
 [ 11.63965846]
 [ 11.63965846]]
